# 피처 엔지니어링

판매자당 상품이 대부분 1개이므로, 집계 통계 대신 개별 값과 리뷰 패턴 기반 피처를 사용합니다.

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 결과 저장 경로 설정
OUTPUT_PATH = '../data/processed/features.csv'
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

print("라이브러리 로드 완료")

## 1. 데이터 로드

In [ ]:
# 데이터 로드
sellers_df = pd.read_csv('../data/processed/ml_sellers.csv')
products_df = pd.read_csv('../data/processed/ml_products.csv')
reviews_df = pd.read_csv('../data/processed/ml_reviews.csv')

print(f"판매자: {len(sellers_df)}개")
print(f"상품: {len(products_df)}개")
print(f"리뷰: {len(reviews_df)}개")

## 2. 개별 상품 피처 (상품 1개인 판매자가 92.8%)

In [ ]:
# 판매자별 대표 상품 피처 (첫 번째 상품 기준)
product_features = products_df.groupby('vendor_name').first().reset_index()

# 평점 분포 비율 계산
product_features['rating_5_ratio'] = product_features['rating_5'] / product_features['review_count'].replace(0, 1)
product_features['rating_4_ratio'] = product_features['rating_4'] / product_features['review_count'].replace(0, 1)
product_features['rating_1_2_ratio'] = (product_features['rating_1'] + product_features['rating_2']) / product_features['review_count'].replace(0, 1)

# 필요한 피처만 선택
product_features = product_features[[
    'vendor_name',
    'price', 'discount_rate', 'product_rating',
    'shipping_fee', 'shipping_days',
    'review_count', 'inquiry_count',
    'rating_5_ratio', 'rating_4_ratio', 'rating_1_2_ratio'
]].rename(columns={'vendor_name': 'company_name'})

print(f"상품 피처 생성: {len(product_features)}개 판매자")
product_features.head()

## 3. 리뷰 패턴 기반 피처

In [ ]:
# 판매자별 리뷰 매핑
product_vendor_map = products_df[['product_id', 'vendor_name']].drop_duplicates()
reviews_with_vendor = reviews_df.merge(product_vendor_map, on='product_id', how='left')

# 리뷰 텍스트 길이
reviews_with_vendor['text_length'] = reviews_with_vendor['review_text'].apply(
    lambda x: len(str(x)) if pd.notna(x) else 0
)

# 짧은 리뷰 여부 (30자 미만)
reviews_with_vendor['is_short_review'] = (reviews_with_vendor['text_length'] < 30).astype(int)

# 5점 리뷰 여부
reviews_with_vendor['is_5_star'] = (reviews_with_vendor['review_rating'] == 5).astype(int)

# 판매자별 리뷰 패턴 통계
review_pattern = reviews_with_vendor.groupby('vendor_name').agg({
    'id': 'count',
    'text_length': 'mean',
    'is_short_review': 'mean',  # 짧은 리뷰 비율
    'is_5_star': 'mean',        # 5점 비율 (리뷰 기준)
    'review_rating': 'mean'
}).reset_index()

review_pattern.columns = [
    'company_name', 'review_count_actual',
    'review_length_mean', 'short_review_ratio', 
    'five_star_ratio', 'review_rating_mean'
]

print(f"리뷰 패턴 피처 생성: {len(review_pattern)}개 판매자")
review_pattern.head()

In [ ]:
# 리뷰 텍스트 유사도 계산 (조작 리뷰 탐지)
def calculate_review_similarity(group):
    """같은 판매자의 리뷰들 간 텍스트 유사도 평균"""
    texts = group['review_text'].dropna().tolist()
    if len(texts) < 2:
        return 0.0
    
    try:
        vectorizer = TfidfVectorizer(max_features=100)
        tfidf_matrix = vectorizer.fit_transform(texts)
        sim_matrix = cosine_similarity(tfidf_matrix)
        
        # 대각선 제외한 평균 유사도
        n = sim_matrix.shape[0]
        total = sim_matrix.sum() - n  # 대각선(자기자신) 제외
        count = n * (n - 1)
        return total / count if count > 0 else 0.0
    except:
        return 0.0

# 판매자별 리뷰 유사도 계산
review_similarity = reviews_with_vendor.groupby('vendor_name').apply(
    calculate_review_similarity, include_groups=False
).reset_index()
review_similarity.columns = ['company_name', 'review_similarity']

print(f"리뷰 유사도 피처 생성: {len(review_similarity)}개 판매자")
print(f"유사도 통계: mean={review_similarity['review_similarity'].mean():.3f}, max={review_similarity['review_similarity'].max():.3f}")

## 4. 피처 병합 및 저장

In [ ]:
# 기본 판매자 정보
features_df = sellers_df[['company_name', 'satisfaction_score', 'is_abusing_seller']].copy()

# 피처 병합
features_df = features_df.merge(product_features, on='company_name', how='left')
features_df = features_df.merge(review_pattern, on='company_name', how='left')
features_df = features_df.merge(review_similarity, on='company_name', how='left')

# 결측치 처리
# - 리뷰가 없는 판매자: 리뷰 관련 피처를 0으로
features_df = features_df.fillna(0)

print(f"최종 피처 데이터: {features_df.shape}")
print(f"\n피처 목록:")
for col in features_df.columns:
    print(f"  - {col}")

features_df.head()

In [ ]:
# 결과 저장
features_df.to_csv(OUTPUT_PATH, index=False)
print(f"피처 데이터 저장 완료: {OUTPUT_PATH}")
print(f"총 {len(features_df)}개 판매자, {len(features_df.columns)-2}개 피처")